In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

import findspark
findspark.init()

In [2]:
import os
for item in sorted(os.environ.items()): print(item)

('Apple_PubSub_Socket_Render', '/private/tmp/com.apple.launchd.zsnS8QhvIw/Render')
('CLICOLOR', '1')
('GIT_PAGER', 'cat')
('HOME', '/Users/williste')
('JPY_PARENT_PID', '736')
('LC_CTYPE', 'UTF-8')
('LOGNAME', 'williste')
('MPLBACKEND', 'module://ipykernel.pylab.backend_inline')
('PAGER', 'cat')
('PATH', '/Users/williste/anaconda3/bin:/Applications/anaconda3/bin:/Library/Frameworks/Python.framework/Versions/3.6/bin:/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin')
('PWD', '/Users/williste')
('PYSPARK_PYTHON', '/Users/williste/anaconda3/bin/python')
('SECURITYSESSIONID', '186a9')
('SHELL', '/bin/bash')
('SHLVL', '2')
('SPARK_HOME', '/usr/local/opt/apache-spark/libexec')
('SSH_AUTH_SOCK', '/private/tmp/com.apple.launchd.DUE9ZWU1zW/Listeners')
('TERM', 'xterm-color')
('TERM_PROGRAM', 'Apple_Terminal')
('TERM_PROGRAM_VERSION', '404')
('TERM_SESSION_ID', '65933B53-CA98-4610-BEB8-AD245DCE3DA5')
('TMPDIR', '/var/folders/dx/60t98k0s1118g99rt_ktqb4w0000gn/T/')
('USER', 

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

import findspark
findspark.init()

#SparkContext.stop(sc)

conf = SparkConf().setMaster("local[100]").setAppName("Spark_4") 
sc = SparkContext(conf=conf)

#SparkContext.setSystemProperty('spark.driver.maxResultSize', '8g') 
#SparkContext.setSystemProperty('spark.executor.memory', '2g')
#SparkContext.setSystemProperty('spark.executor.cores', '4')
#SparkContext.setSystemProperty('spark.memory.fraction', '0.6')
conf.get("spark.app.name")

sc._conf.getAll()
for item in sorted(sc._conf.getAll()): print(item)
    
print(sc.version)

('spark.app.id', 'local-1530277493629')
('spark.app.name', 'Spark_4')
('spark.driver.host', '172.20.10.3')
('spark.driver.port', '52570')
('spark.executor.id', 'driver')
('spark.master', 'local[100]')
('spark.rdd.compress', 'True')
('spark.serializer.objectStreamReset', '100')
('spark.submit.deployMode', 'client')
('spark.ui.showConsoleProgress', 'true')
2.3.1


In [4]:
sqlContext = SQLContext(sc)

df = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Users/williste/Downloads/ny-parking-violations-issued/parking-violations-issued-fiscal-year-2016.csv")

In [12]:
d

+--------------+--------+------------------+----------+-------------------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+-----------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Iss

In [5]:
df_samp = df.sample(False, 0.0001, 42)
df_samp.count()

1089

In [6]:
df_samp.groupBy('Issue Date').count().show()
# Fiskaljahr

+-------------------+-----+
|         Issue Date|count|
+-------------------+-----+
|2017-08-16 00:00:00|    1|
|2015-06-22 00:00:00|    1|
|2016-02-01 00:00:00|    4|
|2016-03-19 00:00:00|    2|
|2015-09-04 00:00:00|    6|
|2016-01-28 00:00:00|    7|
|2016-04-13 00:00:00|    2|
|2015-07-26 00:00:00|    2|
|2015-09-20 00:00:00|    3|
|2015-11-23 00:00:00|    6|
|2015-11-25 00:00:00|    2|
|2016-02-17 00:00:00|    1|
|2016-05-19 00:00:00|    2|
|2015-11-10 00:00:00|    6|
|2015-12-27 00:00:00|    2|
|2016-01-08 00:00:00|    3|
|2016-04-07 00:00:00|    4|
|2015-09-18 00:00:00|    6|
|2016-05-03 00:00:00|    5|
|2015-09-30 00:00:00|    6|
+-------------------+-----+
only showing top 20 rows



In [123]:
df_samp = df_samp.withColumn('am_pm',df_samp['Violation Time'].substr(5, 1))
df_samp = df_samp.withColumn('time',df_samp['Violation Time'].substr(1, 4))
#df_samp.show()
df_samp.groupBy('am_pm').count().show()
df_samp.groupBy('time').count().show()

+-----+-----+
|am_pm|count|
+-----+-----+
|    A|  559|
|    P|  530|
+-----+-----+

+----+-----+
|time|count|
+----+-----+
|0955|    2|
|0851|    5|
|1159|    3|
|0905|    5|
|0833|    4|
|1241|    1|
|0132|    3|
|0452|    1|
|0558|    3|
|0709|    3|
|0836|    5|
|0724|    1|
|0126|    1|
|0319|    2|
|0539|    1|
|0312|    2|
|1043|    2|
|0153|    3|
|0314|    2|
|0545|    1|
+----+-----+
only showing top 20 rows



In [26]:
# Variables and Formats
# df_samp.__dict__
# df_samp.columns
# df_samp.schema.names
# df_samp.printSchema()
# Issue Date: timestamp (nullable = true)

from pyspark.sql.functions import date_format
df_samp.select('Issue Date', 
               date_format('Issue Date', 'u').alias('dow_number'), 
               date_format('Issue Date', 'E').alias('dow_string')).show()

+-------------------+----------+----------+
|         Issue Date|dow_number|dow_string|
+-------------------+----------+----------+
|2015-07-23 00:00:00|         4|       Thu|
|2015-06-28 00:00:00|         7|       Sun|
|2015-06-16 00:00:00|         2|       Tue|
|2015-06-12 00:00:00|         5|       Fri|
|2015-07-08 00:00:00|         3|       Wed|
|2015-07-13 00:00:00|         1|       Mon|
|2015-07-21 00:00:00|         2|       Tue|
|2015-07-24 00:00:00|         5|       Fri|
|2015-06-17 00:00:00|         3|       Wed|
|2015-06-24 00:00:00|         3|       Wed|
|2015-07-02 00:00:00|         4|       Thu|
|2015-07-17 00:00:00|         5|       Fri|
|2015-07-28 00:00:00|         2|       Tue|
|2015-07-27 00:00:00|         1|       Mon|
|2015-07-06 00:00:00|         1|       Mon|
|2015-07-17 00:00:00|         5|       Fri|
|2015-07-14 00:00:00|         2|       Tue|
|2015-07-09 00:00:00|         4|       Thu|
|2015-07-05 00:00:00|         7|       Sun|
|2015-06-27 00:00:00|         6|

In [101]:
import datetime
import calendar
from pyspark.sql.functions import udf

sqlContext.registerFunction("day", lambda x: x.day)
sqlContext.registerDataFrameAsTable(df_samp, "df_samp")
sqlContext.sql("""SELECT `Issue Date`,day(`Issue Date`) FROM df_samp""").show()

+-------------------+---------------+
|         Issue Date|day(Issue Date)|
+-------------------+---------------+
|2015-07-23 00:00:00|             23|
|2015-06-28 00:00:00|             28|
|2015-06-16 00:00:00|             16|
|2015-06-12 00:00:00|             12|
|2015-07-08 00:00:00|              8|
|2015-07-13 00:00:00|             13|
|2015-07-21 00:00:00|             21|
|2015-07-24 00:00:00|             24|
|2015-06-17 00:00:00|             17|
|2015-06-24 00:00:00|             24|
|2015-07-02 00:00:00|              2|
|2015-07-17 00:00:00|             17|
|2015-07-28 00:00:00|             28|
|2015-07-27 00:00:00|             27|
|2015-07-06 00:00:00|              6|
|2015-07-17 00:00:00|             17|
|2015-07-14 00:00:00|             14|
|2015-07-09 00:00:00|              9|
|2015-07-05 00:00:00|              5|
|2015-06-27 00:00:00|             27|
+-------------------+---------------+
only showing top 20 rows



In [6]:
# UDF registered Function in SQL Context
import datetime
import calendar
from pyspark.sql.functions import date_format
from pyspark.sql.functions import udf

def get_weekday(date):
    import datetime
    import calendar  
    weekday = datetime.date(date.year, date.month, date.day)
    return calendar.day_name[weekday.weekday()]
sqlContext.udf.register('get_weekday', get_weekday)

# Test auf Sample
#sqlContext.registerDataFrameAsTable(df_samp, "df_samp")
#sqlContext.sql("""SELECT `Issue Date`, get_weekday(`Issue Date`) as day FROM df_samp""").show()

sqlContext.registerFunction("year", lambda x: x.year)
sqlContext.registerFunction("day", lambda x: x.day)



<function __main__.<lambda>(x)>

In [8]:
sqlContext.registerDataFrameAsTable(df, "df")

df_2 = sqlContext.sql("""
                 SELECT get_weekday(`Issue Date`) as day,
                        year(`Issue Date`) as year      
                   FROM df
                      """)

df_2.show()


+--------+----+
|     day|year|
+--------+----+
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
|Thursday|2015|
+--------+----+
only showing top 20 rows



In [11]:
sqlContext.registerDataFrameAsTable(df_2, "df_2")

#df_2.createOrReplaceTempView("df_view_2")

df_3 = sqlContext.sql("""
                 SELECT day,
                        count(*) as cnt      
                   FROM df_2
                  WHERE  year = 2016
                  GROUP BY day
                     """)

df_3.show()


+---------+------+
|      day|   cnt|
+---------+------+
|Wednesday|831311|
|  Tuesday|893608|
|   Friday|820128|
| Thursday|912995|
| Saturday|471412|
|   Monday|762527|
|   Sunday|180640|
+---------+------+



In [62]:
s1pd = df_samp.toPandas()

In [16]:
print(s1pd)

      Summons Number  Plate ID Registration State Plate Type Issue Date  \
0         1379501222  T649390C                 NY        999 2015-07-23   
1         1381851393   GSD4374                 NY        PAS 2015-06-28   
2         1381938322   GLH9566                 NY        PAS 2015-06-16   
3         1385380100    MH6306                 NY        COM 2015-06-12   
4         4608543234   GWF4448                 NY        PAS 2015-07-08   
5         4608652953    T14ECJ                 NJ        PAS 2015-07-13   
6         4608734568  T636030C                 NY        OMT 2015-07-21   
7         4608838003  T410855C                 NY        OMT 2015-07-24   
8         5086796702   GSZ8247                 NY        PAS 2015-06-17   
9         5086938521    XZ696T                 NJ        PAS 2015-06-24   
10        7000682176   GFE5229                 NY        PAS 2015-07-02   
11        7085142662   GMV7291                 NY        PAS 2015-07-17   
12        7118382991   GV

In [41]:
from pyspark.sql import functions as f
from matplotlib import pyplot as plt 
import seaborn as sns
plt.style.use('fivethirtyeight')

import pandas as pd

samp_pdf = df_samp.toPandas()

In [45]:
samp_pdf['Violation Time']

0       0115P
1       0940P
2       1145A
3       1108A
4       0816A
5       1227P
6       0929A
7       0124P
8       0117P
9       0739P
10      0922A
11      0520P
12      0701A
13      0209P
14      0152P
15      0636A
16      1028A
17      1145A
18      0517P
19      0841A
20      0920A
21      0857A
22      0419P
23      0246P
24      0627A
25      1147A
26      0946A
27      0837A
28      1116P
29      0126P
        ...  
1054    0251A
1055    1229P
1056    0903A
1057    1222P
1058    1149A
1059    0846P
1060    0417P
1061    0937A
1062    0839A
1063    0115P
1064    0913A
1065    1006P
1066    0339P
1067    1055A
1068    0100P
1069    0252P
1070    1112A
1071    1127A
1072    0840A
1073    0137A
1074    1249P
1075    0233P
1076    1255P
1077    0432P
1078    1033A
1079    0942A
1080    1128A
1081    0853A
1082    1129A
1083    0901A
Name: Violation Time, Length: 1084, dtype: object